In [13]:
etats = ('NNP', 'MD','VB','JJ','NN','RB','DT')

proba_0 = {'NNP': 0.2767,'MD': 0.0006,'VB':0.0031,'JJ':0.0453,'NN':0.0449,'RB':0.0510,'DT':0.2026
}

trans_prob = {
    'NNP':{'NNP': 0.3777,'MD': 0.0110,'VB':0.0009,'JJ':0.0084,'NN':0.0584,'RB':0.0090,'DT':0.0025},
    'MD':{'NNP': 0.0008,'MD': 0.0002,'VB':0.7968,'JJ':0.0005,'NN':0.0008,'RB':0.1698,'DT':0.0014},
    'VB':{'NNP': 0.0322,'MD': 0.0005,'VB':0.0050,'JJ':0.0837,'NN':0.0615,'RB':0.0514,'DT':0.2231},
    'JJ':{'NNP': 0.0366,'MD': 0.0004,'VB':0.0001,'JJ':0.0753,'NN':0.4509,'RB':0.0036,'DT':0.0036},
    'NN':{'NNP': 0.0096,'MD': 0.0176,'VB':0.0014,'JJ':0.0086,'NN':0.1216,'RB':0.0177,'DT':0.0068},
    'RB':{'NNP': 0.0068,'MD': 0.0102,'VB':0.1011,'JJ':0.1012,'NN':0.0120,'RB':0.0728,'DT':0.0479},
    'DT':{'NNP': 0.1147,'MD': 0.0021,'VB':0.0002,'JJ':0.2157,'NN':0.4744,'RB':0.0102,'DT':0.0017}
}

vrais_prob = {
    'NNP': {'Janet': 0.000032, 'will': 0.0, 'back': 0.0, 'the': 0.000048, 'bill': 0.0},
    'MD': {'Janet': 0.0, 'will': 0.308431, 'back': 0.0, 'the': 0.0, 'bill': 0.0},
    'VB': {'Janet': 0.0, 'will': 0.000028, 'back': 0.000672, 'the': 0.0, 'bill': 0.000028},
    'JJ': {'Janet': 0.0, 'will': 0.000028, 'back': 0.000340, 'the': 0.0, 'bill': 0.0},
    'NN': {'Janet': 0.0, 'will': 0.000200, 'back': 0.000223, 'the': 0.0, 'bill': 0.002337},
    'RB': {'Janet': 0.0, 'will': 0.0, 'back': 0.010446, 'the': 0.0, 'bill': 0.0},
    'DT': {'Janet': 0.0, 'will': 0.0, 'back': 0.0, 'the': 0.506099, 'bill': 0.0},

}

obser = ['Janet', 'will', 'back', 'the', 'bill']

In [14]:
# Cette fonction permet d'nitialise la matrice V pour stocker les probabilités et les états précédents
def viterbi_initialisation(etats, proba_0, vrais_prob, obs):
    V = [{}] # Crée V comme une liste contenant un dictionnaire pour la première observation
    for i in etats:
        # Pour chaque état, calcule la probabilité initiale
        # et stocke la probabilité et l'état précédent dans V 
        V[0][i] = {"prob": proba_0[i] * vrais_prob[i][obser[0]], "prev": None}  
    return V

In [15]:
# Cette fonction permet de remplit de manière récursive la matrice V pour les observations suivantes   
def viterbi_remplissage_V(V, etats, trans_p, emit_p, obs):
    for t in range(1, len(obser)):
        V.append({}) # Ajoute un nouveau dictionnaire pour l'observation courante
        for i in etats:
            # Trouve la meilleure probabilité de transition depuis les états précédents
            max_prob = max(V[t-1][prev_etiq]["prob"]*trans_prob[prev_etiq][i] for prev_etiq in etats)
            for prev_etiq in etats:
                # Met à jour V avec la nouvelle probabilité et l'état précédent
                if V[t-1][prev_etiq]["prob"] * trans_prob[prev_etiq][i] == max_prob:
                    max_prob = max_prob * vrais_prob[i][obser[t]]
                    V[t][i] = {"prob": max_prob, "prev": prev_etiq}
                    break
    return V

In [16]:
# Cette fonction permet de retrouve le chemin optimal dans V            
def viterbi_output(V):
    opt = [] # la liste qui va contenir le chemin optimal 
    # Trouve la probabilité finale maximale et l'état final correspondant
    max_prob = max(value["prob"] for value in V[-1].values())  
    previous = None
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st) 
            previous = st
            break
    # Retourne en arrière pour trouver les états précédents optimaux
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    return opt
                

In [17]:
#La fonction globale
def viterbi(obser, etats, proba_0, trans_prob, vrais_prob):
    chm_pp={}# dictionnaire de mot avec l'étiquette la plus probable
    V = viterbi_initialisation( etats,  proba_0, vrais_prob, obser)
    V = viterbi_remplissage_V(V, etats, trans_prob, vrais_prob, obser)
    opt = viterbi_output(V)
    for obs,et in zip(obser,opt):
        chm_pp[obs]=et
    return chm_pp

In [18]:
chemin= viterbi(obser, etats, proba_0, trans_prob, vrais_prob)
print("Le chemin le plus probable est : ",chemin)

Le chemin le plus probable est :  {'Janet': 'NNP', 'will': 'MD', 'back': 'VB', 'the': 'DT', 'bill': 'NN'}
